In [ ]:
import scipy.io
import h5py
import pandas as pd
import numpy as np
import os
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/rijkmuseum')

In [ ]:
#Read in labels
labels = pd.read_csv('labels.txt',delimiter = ',',header=None)
names = pd.read_csv('names.txt',delimiter = '/t',header=None)
labels = labels.to_numpy(dtype = 'int')
labels = np.squeeze(labels)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
##Load in Art
numart = labels.shape[0] 

#Create dummy array
artlog = np.zeros((numart,224,224,3),dtype = 'int')

#Get addresses
img_folder = '' #Address of the folder with images goes here. Make sure you write the complete address
img_filenames = os.listdir(img_folder)
img_filenames.sort()
art_addr = []
for ifn in img_filenames:
  art_addr.append(img_folder + '/' + ifn)


#Load in pieces one at a time
for x in range(numart):
  newart = cv2.imread(art_addr[x])
  if len(newart.shape) == 2:
    newart = np.expand_dims(newart,axis=2)
    newart = np.concatenate((newart,newart,newart),axis=2)
  
  artlog[x,:,:,:] = newart





In [ ]:
## Remove Anonymous/Unknown Author Art (If you wish. Would probably help performance but reduces data by 15%)
named_art = (labels <= (names.shape[0] - 2))
artlog = artlog[named_art,:,:,:]
labels = labels[named_art]
numart = labels.shape[0]

In [ ]:
## Remove Art from Artists with fewer than 10 pieces
test_passers = (labels < -2)

for x in np.arange(1,names.shape[0]+1):
  artistnum = np.count_nonzero(labels == x)
  if artistnum >= 50:
    test_passers = np.logical_or(test_passers,(labels == x))

#artlog = artlog[test_passers,:,:,:]
labels = labels[test_passers]
numart = labels.shape[0]


In [ ]:
## Shuffle - Split - Check for Balance
goodbalance = False

while goodbalance == False:
  ## Shuffle

  #Create Shuffle
  mixer = np.arange(numart)
  np.random.shuffle(mixer)

  #Apply Shuffle
  artlog = artlog[mixer,:,:,:]
  labels = labels[mixer]

  ## Split Data

  #Declare Split Fractions
  testfrac = 0.1
  valfrac = 0.1

  #Divide into Splits
  test_train_split = round((1-(testfrac+valfrac)) * labels.shape[0])
  test_val_split = round((1-(valfrac)) * labels.shape[0])

  artlog_train = artlog[:test_train_split,:,:,:]
  artlog_test = artlog[test_train_split:test_val_split,:,:,:]
  artlog_val = artlog[test_val_split:,:,:,:]

  labels_train = labels[:test_train_split]
  labels_test = labels[test_train_split:test_val_split]
  labels_val = labels[test_val_split:]

  ##Check for Balance
  
  check_bool = np.array([],dtype=bool)

  #Perform Artist Count in Each Split
  total_bc = np.bincount(labels)
  train_bc = np.concatenate((np.bincount(labels_train),np.zeros(len(total_bc)-len(np.bincount(labels_train),))),)
  test_bc = np.concatenate((np.bincount(labels_test),np.zeros(len(total_bc)-len(np.bincount(labels_test),))),)
  val_bc = np.concatenate((np.bincount(labels_val),np.zeros(len(total_bc)-len(np.bincount(labels_val),))),)

  #Check balance at each artist
  for i in range(len(total_bc)):
    if (total_bc[i] != 0):
      check_bool = np.append(check_bool,( (abs(train_bc[i]-total_bc[i]*0.8) >= total_bc[i]*0.2) or (abs(test_bc[i]-total_bc[i]*0.1) >= total_bc[i]*0.1) or (abs(val_bc[i]-total_bc[i]*0.1) >= total_bc[i]*0.1 ) ))

  print(sum(check_bool))

  if sum(check_bool) <= 0:
    goodbalance = True


3
4
4
6
7
2
1
1
4
1
3
6
3
4
2
5
2
3
2
2
4
4
5
6
3
3
6
6
3
3
6
2
1
3
6
4
5
2
4
5
7
3
2
1
4
5
4
2
5
6
4
2
2
4
2
4
6
4
5
1
4
4
3
2
5
6
3
7
3
6
8
5
3
6
2
1
3
3
2
4
2
8
5
8
4
7
4
1
1
2
3
3
5
2
4
3
3
4
1
3
5
5
4
4
7
8
2
5
5
4
8
7
1
2
2
4
2
2
2
2
7
5
1
7
3
3
0


In [ ]:
## Direct Name Labels (If you want them)

namelabels_train = names[labels_train]
namelabels_test = names[labels_test]
namelabels_val = names[labels_val]

None
